In [116]:
import os 
import random
import pickle
from copy import deepcopy
from collections import defaultdict
from typing import Literal

import networkx as nx 
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.cluster import AgglomerativeClustering
import solver.ga as ga
from solver.slap_models import SLAP, SLAP_PA
import scipy
from math import ceil
from pulp import *
from warehouse_model.layout import Layout
from warehouse_model.simulation import Order, OrderPickEvent, run_simulation, Agent, s_shape, stichgang
from utils import sort_dict, get_products, prepare_orders, calc_product_affinity, calc_product_freq, render_warehouse
import datetime

# Orders

In [117]:
"""Use this code to concat all orders to generate a large order set"""
orders = []
# Folder containing your text files
folder_path = 'data/orders/large'

# Get a list of all text files in the folder
file_list = [f for f in os.listdir(folder_path)]

# Initialize an empty DataFrame to store the data
combined_data = pd.DataFrame()

# Iterate through each text file and read its contents into a DataFrame
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines[2:]:
        orderline = []
        items = line.strip().split()[1:]
        for i in items[::2]:
            orderline.append(int(i))
        orders.append(orderline)

'Use this code to concat all orders to generate a large order set'

In [118]:
# with open('data/orders/large/instances_d1_ord5', 'r') as file:
#     lines = file.readlines()

# # Extract the total number of products
# num_products = int(lines[0].strip())

# # Initialize a list to store the data
# orders = []

# # Loop through the remaining lines to extract product information
# for line in lines[2:]:
#     orderline = []
#     items = line.strip().split()[1:]
#     print(items)
#     print(items[1::2])
#     items_ordered = [int(i) for i in items[::2]]
#     quantity_orderd = [int(i) for i in items[1::2]]
#     for i, item in enumerate(items_ordered):
#         for _ in range(quantity_orderd[i]):
#             orderline.append(item)
#     orders.append(orderline)

['1', '3', '5', '8', '21', '4']
['3', '8', '4']
['1', '3', '5', '8', '21', '4', '12', '3', '11', '7']
['3', '8', '4', '3', '7']
['1', '3', '5', '8', '21', '4', '12', '3', '13', '7']
['3', '8', '4', '3', '7']
['1', '3', '5', '8', '21', '4', '12', '3', '11', '7']
['3', '8', '4', '3', '7']


In [119]:
products = get_products(orders)

In [120]:
products_mapping = {p: i for i, p in enumerate(products)}

In [121]:
with open('data/orders/product_description', 'r') as file:
    lines = file.readlines()



# Extract the total number of products
num_products = int(lines[0].strip())

# Initialize a list to store the data
master_data = pd.DataFrame()
# Loop through the remaining lines to extract product information
for line in lines[2:]:
    l = line.strip().split(",")
    product_id = int(l[0])
    
    if product_id in products:
        data = {"Item": [products_mapping[product_id]], "Category": [l[1]], "ItemName": [l[5]]}
        row = pd.DataFrame(data)
        master_data = pd.concat([master_data, row])

#master_data.to_csv("master_data.csv", index = False)

In [122]:
orders = prepare_orders(orders, products_mapping)
products = get_products(orders)
n_products = len(products)

In [123]:
orders

[[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2],
 [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4],
 [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5],
 [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4]]

In [124]:
data = []
for order_id, order in enumerate(orders, start=1):
    data.extend([{'OrderID': order_id, 'Item': item} for item in order])

# Create a pandas DataFrame
df_orders = pd.DataFrame(data)


In [125]:
#df_orders.merge(master_data, on="Item", how="left").to_csv("orders_slap_large.csv", index=False)

In [126]:
product_frequency = calc_product_freq(products, orders)

# Layout

In [127]:
n_products = len(products)
n_products

6

In [128]:
x = 3
z = 1
storage_per_aisle = x 
y = ceil(n_products / storage_per_aisle / z) * 2

In [129]:
y

4

In [130]:
double_deep = False 

warehouse = Layout(x+2, y, z, double_deep, False, None)

path = os.path.join("data/cache/dist_mats/", "dist_mat_" + str(warehouse.layout_grid.shape) + "_" + str(double_deep) + ".npy")
if os.path.exists(path):
    dist_mat = np.load(path)
    warehouse.load_dist_mat(dist_mat)
else:
    dist_mat = warehouse.gen_dist_mat(path)

In [131]:
storage_locs = [warehouse.nodes_list.index(loc) for loc in warehouse.storage_locs]

# PF PA 

In [132]:
assignment_heuristic = ga.heuristic_solution(warehouse, product_frequency, n_products, (0,0,0))

In [133]:
#initial_population = [ga.generate_genome(products, warehouse) for i in range(100)]
initial_population = []
for i in range(10):
    gen = deepcopy(assignment_heuristic)
    initial_population.append(gen)

In [134]:
#initial_population = [ga.generate_genome(products, warehouse) for i in range(100)]

In [135]:
product_pairs_frequency = calc_product_affinity(orders)

In [141]:
product_frequency.values()

dict_values([12, 32, 16, 9, 14, 7])

In [142]:
product_frequency_pf_pa = [0 for i in range(n_products)]
for i in list(product_frequency.keys()):
    product_frequency_pf_pa[i] += i 

In [143]:
product_pairs_np = np.zeros((n_products, n_products))

In [144]:
for key in product_pairs_frequency.keys():
    for pair in product_pairs_frequency[key].keys():
        product_pairs_np[key][pair] = product_pairs_frequency[key][pair]

In [161]:
product_frequency_pf_pa = list(product_frequency.values())
fitness_tracker_ = []
min_fitness = np.inf
for g in range(100):
    new_pop = []
    pop_results = []
    
   
    fitness_result = [ga.fitness_pf_pa(pop, dist_mat, product_frequency_pf_pa, product_pairs_frequency) for pop in initial_population]
    print(g, min(fitness_result))
    fitness_tracker_.append(min(fitness_result))
    for _ in range(100):
        selected = ga.selection(initial_population, fitness_result)
        child = ga.crossover(selected)
        mutated = ga.mutation(child)
        new_pop.append(mutated)
    initial_population = new_pop

min_fitness = min(fitness_result)
min_child = initial_population[fitness_result.index(min_fitness)]

0 4858.0
1 4858.0
2 4858.0
3 4858.0
4 4858.0
5 4858.0
6 4858.0
7 4858.0
8 4858.0
9 4858.0
10 4858.0
11 4858.0
12 4858.0
13 4858.0
14 4858.0
15 4858.0
16 4858.0
17 4858.0
18 4858.0
19 4858.0
20 4858.0
21 4858.0
22 4858.0
23 4858.0
24 4858.0
25 4858.0
26 4858.0
27 4858.0
28 4858.0
29 4858.0
30 4858.0
31 4858.0
32 4858.0
33 4858.0
34 4858.0
35 4858.0
36 4858.0
37 4858.0
38 4858.0
39 4858.0
40 4858.0
41 4858.0
42 4858.0
43 4858.0
44 4858.0
45 4858.0
46 4858.0
47 4858.0
48 4858.0
49 4858.0
50 4858.0
51 4858.0
52 4858.0
53 4858.0
54 4858.0
55 4858.0
56 4858.0
57 4858.0
58 4858.0
59 4858.0
60 4858.0
61 4858.0
62 4858.0
63 4858.0
64 4858.0
65 4858.0
66 4858.0
67 4858.0
68 4858.0
69 4858.0
70 4858.0
71 4858.0
72 4858.0
73 4858.0
74 4858.0
75 4858.0
76 4858.0
77 4858.0
78 4858.0
79 4858.0
80 4858.0
81 4858.0
82 4858.0
83 4858.0
84 4858.0
85 4858.0
86 4858.0
87 4858.0
88 4858.0
89 4858.0
90 4858.0
91 4858.0
92 4858.0
93 4858.0
94 4858.0
95 4858.0
96 4858.0
97 4858.0
98 4858.0
99 4858.0


In [ ]:
depot = 0
slap = SLAP(dist_mat, products, product_frequency, storage_locs, depot)

In [ ]:
slap.report()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.6.0 21G816)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 36 columns and 72 nonzeros
Model fingerprint: 0x7eb30d82
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 378.0000000
Presolve time: 0.07s
Presolved: 12 rows, 36 columns, 72 nonzeros
Variable types: 0 continuous, 36 integer (36 binary)

Root relaxation: objective 3.280000e+02, 22 iterations, 0.03 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     328.0000000  328.00000  0.00%     -    0s

Ex

In [156]:
depot = 0
slap = SLAP_PA(dist_mat, products, product_frequency, product_pairs_np, storage_locs, depot)

In [157]:
slap.report()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.6.0 21G816)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 36 columns and 72 nonzeros
Model fingerprint: 0x9e66bbbf
Model has 510 quadratic objective terms
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+02]
  QObjective range [1e+02, 4e+03]
  Bounds range     [1e+00, 1e+00]


  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5286.0000000
Presolve time: 0.01s
Presolved: 522 rows, 546 columns, 1602 nonzeros
Variable types: 0 continuous, 546 integer (546 binary)

Root relaxation: objective 3.280000e+02, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  328.00000    0   12 5286.00000  328.00000  93.8%     -    0s
H    0     0                    5040.0000000  328.00000  93.5%     -    0s
     0     0  365.00000    0   12 5040.00000  365.00000  92.8%     -    0s
H    0     0                    4944.0000000  365.00000  92.6%     -    0s
     0     0  387.00000    0   12 4944.00000  387.00000  92.2%     -    0s
     0     0  634.00000    0   12 4944.00000  634.00000  87.2%     -    0s
     0     2  899.00000    0   12 4944.00000  899.00000  81.8%     -    0s
*   24     6               8

In [158]:
slap.model.objVal

4858.0

# Visualisierung

In [152]:
fig = render_warehouse(warehouse, min_child, product_frequency)

In [153]:
fig.show()